In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
# This is a prototype of parsing minecraft data into tables for analysis


In [7]:
dfbase = spark.read.json("Files/json/20230728-2.json")
# df now is a Spark DataFrame containing JSON data from "Files/json/20230728-2.json".
display(dfbase)

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7f83688a-6ae8-4256-9595-17ca7a13a725)

In [8]:
dfbase.schema

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 10, Finished, Available)

StructType([StructField('type', StringType(), True), StructField('value', StructType([StructField('client_id', StringType(), True), StructField('client_modded', BooleanType(), True), StructField('event_timestamp_utc', StringType(), True), StructField('game_mode', StringType(), True), StructField('game_version', StringType(), True), StructField('launcher_name', StringType(), True), StructField('load_time_bootstrap_ms', LongType(), True), StructField('load_time_loading_overlay_ms', LongType(), True), StructField('load_time_pre_window_ms', LongType(), True), StructField('load_time_total_time_ms', LongType(), True), StructField('minecraft_session_id', StringType(), True), StructField('new_world', BooleanType(), True), StructField('operating_system', StringType(), True), StructField('opt_in', BooleanType(), True), StructField('platform', StringType(), True), StructField('seconds_since_load', LongType(), True), StructField('server_modded', BooleanType(), True), StructField('server_type', Str

In [11]:
from pyspark.sql.types import *
import json

jschemabasetables = '{"fields":[{"metadata":{},"name":"type","nullable":true,"type":"string"},{"metadata":{},"name":"value","nullable":true,"type":"string"}],"type":"struct"}'

schemaabasetables = StructType.fromJson(json.loads(jschemabasetables))

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 13, Finished, Available)

In [12]:
dfbase.write.mode("overwrite").format("delta").option("mergeSchema", "true").option("schema", "schemabasetables").save("abfss://minecraft@onelake.dfs.fabric.microsoft.com/minecraftTelemetry.Lakehouse/Tables/minecraft_base_telemetry")


StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 14, Finished, Available)

In [19]:
# Load game_load_times

from pyspark.sql.types import *
import json

jschemagame_load_times = '{"fields":[{"metadata":{},"name":"user_id","nullable":true,"type":"string"},{"metadata":{},"name":"client_id","nullable":true,"type":"string"},{"metadata":{},"name":"minecraft_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"game_version","nullable":true,"type":"string"},{"metadata":{},"name":"operating_system","nullable":true,"type":"string"},{"metadata":{},"name":"platform","nullable":true,"type":"string"},{"metadata":{},"name":"client_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"launcher_name","nullable":true,"type":"string"},{"metadata":{},"name":"event_timestamp_utc","nullable":true,"type":"string"},{"metadata":{},"name":"opt_in","nullable":true,"type":"boolean"},{"metadata":{},"name":"load_time_total_time_ms","nullable":true,"type":"long"},{"metadata":{},"name":"load_time_pre_window_ms","nullable":true,"type":"long"},{"metadata":{},"name":"load_time_bootstrap_ms","nullable":true,"type":"long"},{"metadata":{},"name":"load_time_loading_overlay_ms","nullable":true,"type":"long"}],"type":"struct"}'

schemajsongame_load_times = StructType.fromJson(json.loads(jschemagame_load_times))

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 21, Finished, Available)

In [14]:
dfglt = spark.sql("SELECT value FROM minecraftTelemetry.minecraft_base_telemetry WHERE type = 'game_load_times'")
display(dfglt)

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, fc9ae304-4a39-4c9e-b9be-fafe3f7a9746)

In [18]:
dfglt.printSchema()

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 20, Finished, Available)

root
 |-- value: struct (nullable = true)
 |    |-- client_id: string (nullable = true)
 |    |-- client_modded: boolean (nullable = true)
 |    |-- event_timestamp_utc: string (nullable = true)
 |    |-- game_mode: string (nullable = true)
 |    |-- game_version: string (nullable = true)
 |    |-- launcher_name: string (nullable = true)
 |    |-- load_time_bootstrap_ms: long (nullable = true)
 |    |-- load_time_loading_overlay_ms: long (nullable = true)
 |    |-- load_time_pre_window_ms: long (nullable = true)
 |    |-- load_time_total_time_ms: long (nullable = true)
 |    |-- minecraft_session_id: string (nullable = true)
 |    |-- new_world: boolean (nullable = true)
 |    |-- operating_system: string (nullable = true)
 |    |-- opt_in: boolean (nullable = true)
 |    |-- platform: string (nullable = true)
 |    |-- seconds_since_load: long (nullable = true)
 |    |-- server_modded: boolean (nullable = true)
 |    |-- server_type: string (nullable = true)
 |    |-- ticks_since_load

In [33]:
dftempglt = dfglt.select('value.user_id','value.client_id', 'value.minecraft_session_id', 'value.game_version','value.operating_system','value.platform', 'value.client_modded', 'value.launcher_name','value.event_timestamp_utc','value.opt_in', 'value.load_time_total_time_ms', 'value.load_time_pre_window_ms','value.load_time_bootstrap_ms','value.load_time_loading_overlay_ms')
display(dftempglt)

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, beedf68c-dbb6-41ce-bb8c-b3d9c3eefdd3)

In [35]:
dftempglt.write.mode("overwrite").format("delta").option("mergeSchema", "true").option("schema", "schemajsongame_load_times").save("abfss://minecraft@onelake.dfs.fabric.microsoft.com/minecraftTelemetry.Lakehouse/Tables/minecraft_game_load_times")

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 37, Finished, Available)

In [37]:
%%sql

SELECT * FROM minecraft_game_load_times

StatementMeta(, c0a561e2-10ee-4292-bf03-9966f0dfe3e2, 39, Finished, Available)

<Spark SQL result set with 1 rows and 14 fields>

In [1]:
# Load world_loaded

from pyspark.sql.types import *
import json

jschemaworld_loaded = '{"fields":[{"metadata":{},"name":"user_id","nullable":true,"type":"string"},{"metadata":{},"name":"client_id","nullable":true,"type":"string"},{"metadata":{},"name":"minecraft_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"game_version","nullable":true,"type":"string"},{"metadata":{},"name":"operating_system","nullable":true,"type":"string"},{"metadata":{},"name":"platform","nullable":true,"type":"string"},{"metadata":{},"name":"client_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"launcher_name","nullable":true,"type":"string"},{"metadata":{},"name":"event_timestamp_utc","nullable":true,"type":"string"},{"metadata":{},"name":"opt_in","nullable":true,"type":"boolean"},{"metadata":{},"name":"world_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"server_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"server_type","nullable":true,"type":"string"},{"metadata":{},"name":"game_mode","nullable":true,"type":"string"}],"type":"struct"}'

schemajsonworld_loaded = StructType.fromJson(json.loads(jschemaworld_loaded))

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 3, Finished, Available)

In [2]:
dfwl = spark.sql("SELECT value FROM minecraftTelemetry.minecraft_base_telemetry WHERE type = 'world_loaded'")
display(dfwl)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, a0d873ef-bae6-4ba8-88b7-917861cb3629)

In [4]:
dftempwl = dfwl.select('value.user_id','value.client_id', 'value.minecraft_session_id', 'value.game_version','value.operating_system','value.platform', 'value.client_modded', 'value.launcher_name','value.event_timestamp_utc','value.opt_in', 'value.world_session_id', 'value.server_modded','value.server_type','value.game_mode')
display(dftempwl)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5ea50a58-8e8e-4040-8332-fe423c078cd7)

In [5]:
dftempwl.write.mode("overwrite").format("delta").option("mergeSchema", "true").option("schema", "schemajsonworld_loaded").save("abfss://minecraft@onelake.dfs.fabric.microsoft.com/minecraftTelemetry.Lakehouse/Tables/minecraft_world_loaded")

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 7, Finished, Available)

In [6]:
%%sql

SELECT * FROM minecraft_world_loaded

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 8, Finished, Available)

<Spark SQL result set with 1 rows and 14 fields>

In [8]:
# Load world_load_times

from pyspark.sql.types import *
import json

jschemaworld_load_times = '{"fields":[{"metadata":{},"name":"user_id","nullable":true,"type":"string"},{"metadata":{},"name":"client_id","nullable":true,"type":"string"},{"metadata":{},"name":"minecraft_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"game_version","nullable":true,"type":"string"},{"metadata":{},"name":"operating_system","nullable":true,"type":"string"},{"metadata":{},"name":"platform","nullable":true,"type":"string"},{"metadata":{},"name":"client_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"launcher_name","nullable":true,"type":"string"},{"metadata":{},"name":"event_timestamp_utc","nullable":true,"type":"string"},{"metadata":{},"name":"opt_in","nullable":true,"type":"boolean"},{"metadata":{},"name":"world_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"server_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"server_type","nullable":true,"type":"string"},{"metadata":{},"name":"world_load_time_ms","nullable":true,"type":"long"},{"metadata":{},"name":"new_world","nullable":true,"type":"boolean"}],"type":"struct"}'

schemajsonworld_load_times = StructType.fromJson(json.loads(jschemaworld_load_times))

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 10, Finished, Available)

In [9]:
dfwlt = spark.sql("SELECT value FROM minecraftTelemetry.minecraft_base_telemetry WHERE type = 'world_load_times'")
display(dfwlt)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, f8721d3c-f686-4045-adc9-8b27490a6d89)

In [10]:
dftempwlt = dfwlt.select('value.user_id','value.client_id', 'value.minecraft_session_id', 'value.game_version','value.operating_system','value.platform', 'value.client_modded', 'value.launcher_name','value.event_timestamp_utc','value.opt_in', 'value.world_session_id', 'value.server_modded','value.server_type','value.world_load_time_ms','value.new_world')
display(dftempwlt)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7bc81b22-2a13-445e-91c4-507be9100c84)

In [11]:
dftempwlt.write.mode("overwrite").format("delta").option("mergeSchema", "true").option("schema", "schemajsonworld_load_times").save("abfss://minecraft@onelake.dfs.fabric.microsoft.com/minecraftTelemetry.Lakehouse/Tables/minecraft_world_load_times")

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 13, Finished, Available)

In [12]:
%%sql

SELECT * FROM minecraft_world_load_times

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 14, Finished, Available)

<Spark SQL result set with 1 rows and 15 fields>

In [13]:
# Load world_unloaded

from pyspark.sql.types import *
import json

jschemaworld_unloaded = '{"fields":[{"metadata":{},"name":"user_id","nullable":true,"type":"string"},{"metadata":{},"name":"client_id","nullable":true,"type":"string"},{"metadata":{},"name":"minecraft_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"game_version","nullable":true,"type":"string"},{"metadata":{},"name":"operating_system","nullable":true,"type":"string"},{"metadata":{},"name":"platform","nullable":true,"type":"string"},{"metadata":{},"name":"client_modded","nullable":false,"type":"boolean"},{"metadata":{},"name":"launcher_name","nullable":true,"type":"string"},{"metadata":{},"name":"event_timestamp_utc","nullable":true,"type":"string"},{"metadata":{},"name":"opt_in","nullable":true,"type":"boolean"},{"metadata":{},"name":"world_session_id","nullable":true,"type":"string"},{"metadata":{},"name":"server_modded","nullable":true,"type":"boolean"},{"metadata":{},"name":"server_type","nullable":true,"type":"string"},{"metadata":{},"name":"seconds_since_load","nullable":true,"type":"long"},{"metadata":{},"name":"ticks_since_load","nullable":true,"type":"long"}],"type":"struct"}'

schemajsonworld_unloaded = StructType.fromJson(json.loads(jschemaworld_unloaded))

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 15, Finished, Available)

In [14]:
dfwu = spark.sql("SELECT value FROM minecraftTelemetry.minecraft_base_telemetry WHERE type = 'world_unloaded'")
display(dfwu)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2fec9d43-6d43-4d80-b8b6-047db470059e)

In [15]:
dftempwu = dfwu.select('value.user_id','value.client_id', 'value.minecraft_session_id', 'value.game_version','value.operating_system','value.platform', 'value.client_modded', 'value.launcher_name','value.event_timestamp_utc','value.opt_in', 'value.world_session_id', 'value.server_modded','value.server_type','value.seconds_since_load','value.ticks_since_load')
display(dftempwu)

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5121f254-54a2-4437-a629-aab931d34fc8)

In [16]:
dftempwu.write.mode("overwrite").format("delta").option("mergeSchema", "true").option("schema", "schemajsonworld_unloaded").save("abfss://minecraft@onelake.dfs.fabric.microsoft.com/minecraftTelemetry.Lakehouse/Tables/minecraft_world_unloaded")

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 18, Finished, Available)

In [17]:
%%sql

SELECT * FROM minecraft_world_unloaded

StatementMeta(, 37043901-f6d0-462a-a343-dae1e06a9e35, 19, Finished, Available)

<Spark SQL result set with 1 rows and 15 fields>